In [4]:
import pandas as pd
import numpy as np

In [5]:
df = pd.read_csv("../input/all-news/articles3.csv")

In [7]:
import string
import re

# some basic data cleaning
translator=str.maketrans('','',string.punctuation)
df.content = df.content.apply(lambda x : str(x).lower())
df.content = df.content.apply(lambda x : x.translate(translator))
df.content = df.content.apply(lambda x : re.sub( '\s+', ' ', x ).strip())

# D2V Word Embeddings

In [ ]:
from gensim import models
sentences = df.content.values.tolist()

doc2vecSentence = [models.doc2vec.LabeledSentence(words=sentences[x], tags=["DOC_"+str(x)]) for x in range(0,len(sentences)) ]
model = models.Doc2Vec(dbow_words =1,
                        min_count = 50, 
                        epochs = 40)
model.build_vocab(doc2vecSentence)

for epoch in range(40):
    print("Training for epoch", epoch)
    model.train(doc2vecSentence, total_examples=len(doc2vecSentence), epochs=40)
    model.alpha -= 0.002  # decrease the learning rate`
    model.min_alpha = model.alpha  # fix the learning rate, no decay

C:\ProgramData\Anaconda3\lib\site-packages\gensim\utils.py:1212: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: DeprecationWarning: Call to deprecated `LabeledSentence` (Class will be removed in 4.0.0, use TaggedDocument instead).
  after removing the cwd from sys.path.


Training for epoch 0
Training for epoch 1


In [ ]:
model.save("model_doc2vec_40epochs")

Create vectors

In [12]:
# get vector data
# X = model[model.wv.vocab]
X = list(model.docvecs.doctag_syn0)

# Approach 1 : embeddings  + kmeans

Here we will :
1. Create document embeddings
2. Cluster using KMeans 

In [13]:
from sklearn.cluster import KMeans
from sklearn import metrics
kmeans = KMeans(n_clusters=10)
kmeans.fit(X)
 
labels = kmeans.labels_
centroids = kmeans.cluster_centers_
 
print ("Cluster id labels for inputted data")
print (labels)
print ("Centroids data")
print (centroids)
 
print ("Score (Opposite of the value of X on the K-means objective which is Sum of distances of samples to their closest cluster center):")
print (kmeans.score(X))
 
silhouette_score = metrics.silhouette_score(X, labels, metric='euclidean')
 
print ("Silhouette_score: ")
print (silhouette_score)

In [15]:
## Assign the labels
df["clust"] = labels

In [22]:
# Check count of news articles in each cluster
df.groupby("clust" )["content"].count().plot(kind = "pie", autopct='%.2f', figsize = (10, 10))